## 1.Forward Propagation

In [1]:
import numpy as np
import pandas as pd
data = {'cgpa': [8.5, 9.2, 7.8], 'profile_score': [85, 92, 78], 'lpa': [10, 12, 8]}
df = pd.DataFrame(data)
X = df[['cgpa', 'profile_score']].values

In [7]:
df

,cgpa,profile_score,lpa
0,8.5,85,10
1,9.2,92,12
2,7.8,78,8


In [6]:
X

array([[ 8.5, 85. ],
       [ 9.2, 92. ],
       [ 7.8, 78. ]])

In [4]:
def initialize_parameters():
  np.random.seed(1)
  w = np.random.randn(2,1)*0.01
  b = np.zeros((1,1))
  return w,b


def forward_propagation(x,w,b):
  z = np.dot(x,w)+b
  a = 1/(1+np.exp(-z))
  return a

In [8]:
w,b = initialize_parameters()
A = forward_propagation(X,w,b)
print("final_output",A)

final_output [[0.40566303]
 [0.39810287]
 [0.41326819]]


## 2.Backward Propagation

In [12]:
import numpy as np

class NeuralNetwork:
  def __init__(self,input_size,hidden_size,output_size):
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
    self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)

    self.bias_hidden = np.zeros((1, self.hidden_size))
    self.bias_output = np.zeros((1, self.output_size))

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))

  def sigmoid_derivative(self,x):
    return x*(1-x)

  def feedforward(self,x):

    self.hidden_activation = np.dot(X, self.weights_input_hidden) + self.bias_hidden
    self.hidden_output = self.sigmoid(self.hidden_activation)

    self.output_activation = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
    self.predicted_output = self.sigmoid(self.output_activation)

    return self.predicted_output

  def backward(self,x,y,learning_rate):

    output_error = y - self.predicted_output
    output_delta = output_error * self.sigmoid_derivative(self.predicted_output)

    hidden_error = np.dot(output_delta,self.weights_hidden_output.T)
    hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_output)

    self.weights_hidden_output += np.dot(self.hidden_output.T, output_delta) * learning_rate

    self.weights_input_hidden += np.dot(X.T, hidden_delta) * learning_rate

    self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate

  def train(self,x,y,epochs,learning_rate):
    for epoch in range(epochs):
      output = self.feedforward(x)
      self.backward(x,y,learning_rate)
      if epochs % 4000 == 0:
        loss = np.mean(np.square(y - output))
        print(f"Epoch {epoch}, Loss: {loss}")

In [13]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1)
nn.train(X, y, epochs=10000, learning_rate=0.1)

output = nn.feedforward(X)
print("Predictions after training:")
print(output)

Predictions after training:
[[0.07742489]
 [0.92753903]
 [0.9383886 ]
 [0.06501285]]


## Train a ANN On churn Modeling

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shrutimechlearn/churn-modelling")

print("Path to dataset files:", path)

100%|██████████| 262k/262k [00:00<00:00, 597kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/shrutimechlearn/churn-modelling/versions/1


In [2]:
import os

data_path = "/root/.cache/kagglehub/datasets/shrutimechlearn/churn-modelling/versions/1"
os.listdir(data_path)


['Churn_Modelling.csv']

In [4]:
import pandas as pd

file_path = f"{data_path}/Churn_Modelling.csv"
df = pd.read_csv(file_path)

# Show first few rows
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [7]:
df.shape

(10000, 14)

In [8]:
x = df.iloc[:,3:13].values
y = df.iloc[:,13].values

In [9]:
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [10]:
y

array([1, 0, 1, ..., 1, 1, 0])

In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

labelencoder_x1 = LabelEncoder()
x[:,1] = labelencoder_x1.fit_transform(x[:,1])

labelencoder_x2 = LabelEncoder()
x[:,2] = labelencoder_x2.fit_transform(x[:,2])

geography_column_encoded = x[:, 1].reshape(-1, 1) # OneHotEncoder expects a 2D array

onehotencoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse_output=False for a dense array

# Fit and transform the geography column
encoded_geography_features = onehotencoder.fit_transform(geography_column_encoded)
encoded_geography_features = encoded_geography_features[:, 1:]

credit_score_column = x[:, 0].reshape(-1, 1)

# Extract Gender column (original index 2, now label-encoded)
gender_column_encoded = x[:, 2].reshape(-1, 1)

# Extract remaining numerical columns (original indices 3 to 9)
other_numerical_columns = x[:, 3:]
x = np.concatenate((encoded_geography_features, gender_column_encoded, credit_score_column, other_numerical_columns), axis=1)


In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

In [15]:
#feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [16]:
# building model
import keras
from keras.models import Sequential
from keras.layers import Dense


In [22]:
classifier = Sequential()
# adding the input layer and first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation='relu', input_dim = 11))
# adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation='relu'))
#adding out put layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation='sigmoid'))
# Compile the ANN
classifier.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

# Fit the ANN to the training set
classifier.fit(x_train,y_train,batch_size = 10, epochs = 20) # add more epochs for good output

Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7908 - loss: 0.5578
Epoch 2/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7974 - loss: 0.4266
Epoch 3/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7996 - loss: 0.4176
Epoch 4/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8292 - loss: 0.4108
Epoch 5/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8230 - loss: 0.4216
Epoch 6/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8254 - loss: 0.4267
Epoch 7/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8369 - loss: 0.4020
Epoch 8/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8271 - loss: 0.4153
Epoch 9/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8333 - loss: 0.4123
Epoch 10/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8318 - loss: 0.4095
Epoch 11/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8408 - loss: 0.3988
Epoch 12/20
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

In [24]:
# predict the test result
y_pred = classifier.predict(x_test)
y_pred = (y_pred > 0.5)
y_pred

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [27]:
# confusion matrics
from sklearn.metrics import confusion_matrix
confusion_metrics = confusion_matrix(y_test,y_pred)
confusion_metrics

array([[1550,   45],
       [ 270,  135]])

In [29]:
# Accuracy Score
from sklearn.metrics import accuracy_score
accuracy_score = accuracy_score(y_test,y_pred)
accuracy_score

0.8425